In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet18_cifar100.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01
mean = [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std = [0.2673342858792401, 0.2564384629170883, 0.27615047132568404]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = torchvision.datasets.CIFAR100(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR100(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=100):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [6]:
checkpoint = 'checkpoint_resnet18_cifar100.pth.tar'
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = ResNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_resnet18_cifar100.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_resnet18_cifar100.txt", "a")
    f2.write('Resnet+cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_resnet18_cifar100.txt", "a")
        f3.write('Resnet+cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_resnet18_cifar100.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()


Loaded checkpoint from epoch 131.

Epoch: [131][0/391]	Batch Time 19.900 (19.900)	Data Time 0.000 (0.000)	Loss 0.0064 (0.0064)	
Epoch: [131][20/391]	Batch Time 0.172 (1.116)	Data Time 0.000 (0.000)	Loss 0.0069 (0.0069)	
Epoch: [131][40/391]	Batch Time 0.172 (0.657)	Data Time 0.000 (0.000)	Loss 0.0077 (0.0073)	
Epoch: [131][60/391]	Batch Time 0.172 (0.499)	Data Time 0.000 (0.000)	Loss 0.0052 (0.0073)	
Epoch: [131][80/391]	Batch Time 0.172 (0.420)	Data Time 0.000 (0.000)	Loss 0.0058 (0.0071)	
Epoch: [131][100/391]	Batch Time 0.172 (0.371)	Data Time 0.000 (0.000)	Loss 0.0063 (0.0071)	
Epoch: [131][120/391]	Batch Time 0.187 (0.339)	Data Time 0.000 (0.000)	Loss 0.0084 (0.0073)	
Epoch: [131][140/391]	Batch Time 0.172 (0.316)	Data Time 0.000 (0.000)	Loss 0.0071 (0.0073)	
Epoch: [131][160/391]	Batch Time 0.187 (0.299)	Data Time 0.000 (0.000)	Loss 0.0054 (0.0071)	
Epoch: [131][180/391]	Batch Time 0.175 (0.285)	Data Time 0.000 (0.000)	Loss 0.0085 (0.0071)	
Epoch: [131][200/391]	Batch Time 0.172

Epoch: [135][140/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 0.0060 (0.0070)	
Epoch: [135][160/391]	Batch Time 0.172 (0.195)	Data Time 0.000 (0.000)	Loss 0.0071 (0.0069)	
Epoch: [135][180/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 0.0073 (0.0070)	
Epoch: [135][200/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.0064 (0.0070)	
Epoch: [135][220/391]	Batch Time 0.172 (0.191)	Data Time 0.000 (0.000)	Loss 0.0095 (0.0070)	
Epoch: [135][240/391]	Batch Time 0.187 (0.190)	Data Time 0.000 (0.000)	Loss 0.0063 (0.0070)	
Epoch: [135][260/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0050 (0.0070)	
Epoch: [135][280/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 0.0046 (0.0070)	
Epoch: [135][300/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.0063 (0.0070)	
Epoch: [135][320/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0057 (0.0069)	
Epoch: [135][340/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)

Epoch: [139][280/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0069 (0.0067)	
Epoch: [139][300/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0043 (0.0067)	
Epoch: [139][320/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.0091 (0.0067)	
Epoch: [139][340/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0069 (0.0067)	
Epoch: [139][360/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0057 (0.0068)	
Epoch: [139][380/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.0054 (0.0069)	
tensor(0.9996)
测试分类准确率为：75.330%
Epoch: [140][0/391]	Batch Time 2.765 (2.765)	Data Time 0.000 (0.000)	Loss 0.0067 (0.0067)	
Epoch: [140][20/391]	Batch Time 0.172 (0.302)	Data Time 0.000 (0.000)	Loss 0.0098 (0.0064)	
Epoch: [140][40/391]	Batch Time 0.187 (0.242)	Data Time 0.000 (0.000)	Loss 0.0048 (0.0066)	
Epoch: [140][60/391]	Batch Time 0.172 (0.221)	Data Time 0.000 (0.000)	Loss 0.0075 (0.0066)	
Epoch: [140][80/391]	Batch Time 0.172 (0.21

Epoch: [144][20/391]	Batch Time 0.187 (0.304)	Data Time 0.000 (0.000)	Loss 0.0046 (0.0064)	
Epoch: [144][40/391]	Batch Time 0.172 (0.242)	Data Time 0.000 (0.000)	Loss 0.0056 (0.0069)	
Epoch: [144][60/391]	Batch Time 0.172 (0.222)	Data Time 0.000 (0.000)	Loss 0.0061 (0.0069)	
Epoch: [144][80/391]	Batch Time 0.172 (0.212)	Data Time 0.000 (0.000)	Loss 0.0067 (0.0068)	
Epoch: [144][100/391]	Batch Time 0.187 (0.205)	Data Time 0.000 (0.000)	Loss 0.0072 (0.0068)	
Epoch: [144][120/391]	Batch Time 0.187 (0.201)	Data Time 0.000 (0.000)	Loss 0.0052 (0.0068)	
Epoch: [144][140/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 0.0035 (0.0068)	
Epoch: [144][160/391]	Batch Time 0.187 (0.195)	Data Time 0.000 (0.000)	Loss 0.0031 (0.0069)	
Epoch: [144][180/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 0.0234 (0.0070)	
Epoch: [144][200/391]	Batch Time 0.197 (0.193)	Data Time 0.000 (0.000)	Loss 0.0059 (0.0069)	
Epoch: [144][220/391]	Batch Time 0.172 (0.191)	Data Time 0.000 (0.000)	Los

Epoch: [148][180/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 0.0044 (0.0069)	
Epoch: [148][200/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 0.0208 (0.0070)	
Epoch: [148][220/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0037 (0.0070)	
Epoch: [148][240/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0051 (0.0069)	
Epoch: [148][260/391]	Batch Time 0.187 (0.190)	Data Time 0.000 (0.000)	Loss 0.0062 (0.0069)	
Epoch: [148][280/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0055 (0.0069)	
Epoch: [148][300/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.0061 (0.0069)	
Epoch: [148][320/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.0063 (0.0070)	
Epoch: [148][340/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.0050 (0.0070)	
Epoch: [148][360/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.0062 (0.0069)	
Epoch: [148][380/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)

...

Epoch: [178][360/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.0087 (0.0068)	
Epoch: [178][380/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.0075 (0.0068)	
tensor(0.9997)
测试分类准确率为：75.310%
Epoch: [179][0/391]	Batch Time 2.757 (2.757)	Data Time 0.000 (0.000)	Loss 0.0057 (0.0057)	
Epoch: [179][20/391]	Batch Time 0.172 (0.302)	Data Time 0.000 (0.000)	Loss 0.0058 (0.0066)	
Epoch: [179][40/391]	Batch Time 0.187 (0.243)	Data Time 0.000 (0.000)	Loss 0.0068 (0.0068)	
Epoch: [179][60/391]	Batch Time 0.172 (0.222)	Data Time 0.000 (0.000)	Loss 0.0044 (0.0066)	
Epoch: [179][80/391]	Batch Time 0.187 (0.211)	Data Time 0.000 (0.000)	Loss 0.0052 (0.0065)	
Epoch: [179][100/391]	Batch Time 0.187 (0.205)	Data Time 0.000 (0.000)	Loss 0.0050 (0.0065)	
Epoch: [179][120/391]	Batch Time 0.187 (0.201)	Data Time 0.000 (0.000)	Loss 0.0054 (0.0064)	
Epoch: [179][140/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 0.0051 (0.0063)	
Epoch: [179][160/391]	Batch Time 0.187 (0.19

Epoch: [183][100/391]	Batch Time 0.187 (0.205)	Data Time 0.000 (0.000)	Loss 0.0116 (0.0067)	
Epoch: [183][120/391]	Batch Time 0.187 (0.201)	Data Time 0.000 (0.000)	Loss 0.0051 (0.0066)	
Epoch: [183][140/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 0.0049 (0.0065)	
Epoch: [183][160/391]	Batch Time 0.187 (0.195)	Data Time 0.000 (0.000)	Loss 0.0058 (0.0066)	
Epoch: [183][180/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 0.0054 (0.0066)	
Epoch: [183][200/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 0.0079 (0.0066)	
Epoch: [183][220/391]	Batch Time 0.172 (0.191)	Data Time 0.000 (0.000)	Loss 0.0057 (0.0067)	
Epoch: [183][240/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0042 (0.0066)	
Epoch: [183][260/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 0.0049 (0.0066)	
Epoch: [183][280/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 0.0056 (0.0066)	
Epoch: [183][300/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)

Epoch: [187][240/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0058 (0.0067)	
Epoch: [187][260/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0051 (0.0068)	
Epoch: [187][280/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 0.0046 (0.0068)	
Epoch: [187][300/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.0070 (0.0068)	
Epoch: [187][320/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0104 (0.0068)	
Epoch: [187][340/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0060 (0.0068)	
Epoch: [187][360/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0075 (0.0067)	
Epoch: [187][380/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 0.0051 (0.0067)	
tensor(0.9997)
测试分类准确率为：75.430%
Epoch: [188][0/391]	Batch Time 2.843 (2.843)	Data Time 0.000 (0.000)	Loss 0.0041 (0.0041)	
Epoch: [188][20/391]	Batch Time 0.187 (0.307)	Data Time 0.000 (0.000)	Loss 0.0062 (0.0060)	
Epoch: [188][40/391]	Batch Time 0.187 (0.

Epoch: [191][380/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 0.0054 (0.0066)	
tensor(0.9997)
测试分类准确率为：75.360%
Epoch: [192][0/391]	Batch Time 2.796 (2.796)	Data Time 0.000 (0.000)	Loss 0.0056 (0.0056)	
Epoch: [192][20/391]	Batch Time 0.172 (0.303)	Data Time 0.000 (0.000)	Loss 0.0042 (0.0055)	
Epoch: [192][40/391]	Batch Time 0.187 (0.243)	Data Time 0.000 (0.000)	Loss 0.0080 (0.0062)	
Epoch: [192][60/391]	Batch Time 0.172 (0.222)	Data Time 0.000 (0.000)	Loss 0.0063 (0.0064)	
Epoch: [192][80/391]	Batch Time 0.187 (0.212)	Data Time 0.000 (0.000)	Loss 0.0066 (0.0063)	
Epoch: [192][100/391]	Batch Time 0.172 (0.205)	Data Time 0.000 (0.000)	Loss 0.0081 (0.0063)	
Epoch: [192][120/391]	Batch Time 0.187 (0.201)	Data Time 0.000 (0.000)	Loss 0.0088 (0.0062)	
Epoch: [192][140/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 0.0083 (0.0064)	
Epoch: [192][160/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 0.0046 (0.0063)	
Epoch: [192][180/391]	Batch Time 0.187 (0.19

Epoch: [196][120/391]	Batch Time 0.172 (0.201)	Data Time 0.000 (0.000)	Loss 0.0055 (0.0066)	
Epoch: [196][140/391]	Batch Time 0.187 (0.198)	Data Time 0.000 (0.000)	Loss 0.0055 (0.0065)	
Epoch: [196][160/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 0.0076 (0.0065)	
Epoch: [196][180/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 0.0047 (0.0065)	
Epoch: [196][200/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 0.0066 (0.0065)	
Epoch: [196][220/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.0062 (0.0065)	
Epoch: [196][240/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0048 (0.0066)	
Epoch: [196][260/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0076 (0.0066)	
Epoch: [196][280/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 0.0070 (0.0067)	
Epoch: [196][300/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 0.0035 (0.0067)	
Epoch: [196][320/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)